# Identify Fraud from Enron Email

## Introduction

> In 2000, Enron was one of the largest companies in the United States. By 2002, it had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, a significant amount of typically confidential information entered into the public record, including tens of thousands of emails and detailed financial data for top executives. This project is to build a person of interest identifier based on financial and email data made public as a result of the Enron scandal.

> A Person of Interest(PoI) is an individual who meets one of the following criteria:
>* Individuals who were indicted
>* Individuals who reached a settlement or plea deal with the government
>* Individuals who testified in exchange for prosecution immunity.

> The full project and all data for this project can be seen in [this Github repository](https://github.com/TrikerDev/Identify-Fraud-from-Enron-Email). Data such as financial records, emails from employees, PoI names, etc, can be seen [here](https://github.com/TrikerDev/Identify-Fraud-from-Enron-Email/tree/master/Identify%20Fraud%20from%20Enron%20Email/Enron%20Data).

## Data Exploration

In [55]:
# Importing packages
import sys
import pickle
sys.path.append("Enron Data/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

In [56]:
# Loading data
with open("Enron Data/final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [57]:
# The number of executives in the dataset
len(data_dict)

146

In [58]:
# The names of the executives
print data_dict.keys()

['METTS MARK', 'BAXTER JOHN C', 'ELLIOTT STEVEN', 'CORDES WILLIAM R', 'HANNON KEVIN P', 'MORDAUNT KRISTINA M', 'MEYER ROCKFORD G', 'MCMAHON JEFFREY', 'HORTON STANLEY C', 'PIPER GREGORY F', 'HUMPHREY GENE E', 'UMANOFF ADAM S', 'BLACHMAN JEREMY M', 'SUNDE MARTIN', 'GIBBS DANA R', 'LOWRY CHARLES P', 'COLWELL WESLEY', 'MULLER MARK S', 'JACKSON CHARLENE R', 'WESTFAHL RICHARD K', 'WALTERS GARETH W', 'WALLS JR ROBERT H', 'KITCHEN LOUISE', 'CHAN RONNIE', 'BELFER ROBERT', 'SHANKMAN JEFFREY A', 'WODRASKA JOHN', 'BERGSIEKER RICHARD P', 'URQUHART JOHN A', 'BIBI PHILIPPE A', 'RIEKER PAULA H', 'WHALEY DAVID A', 'BECK SALLY W', 'HAUG DAVID L', 'ECHOLS JOHN B', 'MENDELSOHN JOHN', 'HICKERSON GARY J', 'CLINE KENNETH W', 'LEWIS RICHARD', 'HAYES ROBERT E', 'MCCARTY DANNY J', 'KOPPER MICHAEL J', 'LEFF DANIEL P', 'LAVORATO JOHN J', 'BERBERIAN DAVID', 'DETMERING TIMOTHY J', 'WAKEHAM JOHN', 'POWERS WILLIAM', 'GOLD JOSEPH', 'BANNANTINE JAMES M', 'DUNCAN JOHN H', 'SHAPIRO RICHARD S', 'SHERRIFF JOHN R', 'SHELBY 

## Outlier Removal

> Right away, we can see some outliers in this dataset. The first is 'THE TRAVEL AGENCY IN THE PARK'. This is supposed to be a list of the names of executives, so this is clearly in the wrong place. Another outlier is 'TOTAL'. This 'TOTAL' data is the sum of all other executives. We dont want the total of every person on the list, so this is an outlier that will be removed

In [59]:
# Removing THE TRAVEL AGENCY IN THE PARK outlier
data_dict.pop('THE TRAVEL AGENCY IN THE PARK', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

In [60]:
# Removing TOTAL outlier
data_dict.pop('TOTAL', 0)

{'bonus': 97343619,
 'deferral_payments': 32083396,
 'deferred_income': -27992891,
 'director_fees': 1398517,
 'email_address': 'NaN',
 'exercised_stock_options': 311764000,
 'expenses': 5235198,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 83925000,
 'long_term_incentive': 48521928,
 'other': 42667589,
 'poi': False,
 'restricted_stock': 130322299,
 'restricted_stock_deferred': -7576788,
 'salary': 26704229,
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 309886585,
 'total_stock_value': 434509511}

> Another way to find potential outliers is to find individuals who may have Total Payments as NaN. This could potentially mean that this person should not be included.

In [61]:
# List of people with no total payment data
for entry in data_dict:
    if data_dict[entry]['total_payments'] == 'NaN':
        print entry

CORDES WILLIAM R
LOWRY CHARLES P
CHAN RONNIE
WHALEY DAVID A
CLINE KENNETH W
LEWIS RICHARD
MCCARTY DANNY J
POWERS WILLIAM
PIRO JIM
WROBEL BRUCE
MCDONALD REBECCA
SCRIMSHAW MATTHEW
GATHMANN WILLIAM D
GILLIS JOHN
MORAN MICHAEL P
LOCKHART EUGENE E
SHERRICK JEFFREY B
FOWLER PEGGY
CHRISTODOULOU DIOMEDES
HUGHES JAMES A
HAYSLETT RODERICK J


> This could be narrowed down further by adding another main criteria to the list: the stock options.

In [62]:
# List of people with no total payment data and no stock option data
for entry in data_dict:
    if data_dict[entry]['total_payments'] == 'NaN' and data_dict[entry]['total_stock_value'] == 'NaN':
        print entry

CHAN RONNIE
POWERS WILLIAM
LOCKHART EUGENE E


> Investigating these individuals, we can see that they are missing lots of information.

In [63]:
# Investigating CHAN RONNIE
data_dict['CHAN RONNIE']

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': -98784,
 'director_fees': 98784,
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 32460,
 'restricted_stock_deferred': -32460,
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}

In [64]:
# Investigating POWERS WILLIAM
data_dict['POWERS WILLIAM']

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': -17500,
 'director_fees': 17500,
 'email_address': 'ken.powers@enron.com',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 26,
 'from_poi_to_this_person': 0,
 'from_this_person_to_poi': 0,
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 12,
 'to_messages': 653,
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}

In [65]:
# Investigating LOCKHART EUGENE E
data_dict['LOCKHART EUGENE E']

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}

> LOCKHART EUGENE E is the most obvious outlier here, as this individual has NaN for every feature, and he is also not a PoI. This is an outlier that will be removed.

> CHAN RONNIE is also an outlier, as he as NaN for most features and is not a PoI. The features he does have are payments and stock. However, these are both cancelled out to zero as they are deferred for the exact same amount. This leaves us with another blank slate. This outlier will be removed.

> POWERS WILLIAM does have NaN and 0 for many of the features, however he does have several features that the other two did not have, such as messages and receipts. This gives up more information that can be used in the investigation. So even though much information is missing, we can still gather some data from this individual. This is not an outlier.

In [66]:
# Removing LOCKHART EUGENE E outlier
data_dict.pop('LOCKHART EUGENE E', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}

In [67]:
# Removing CHAN RONNIE outlier
data_dict.pop('CHAN RONNIE', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': -98784,
 'director_fees': 98784,
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 32460,
 'restricted_stock_deferred': -32460,
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}

In [68]:
# Saving this new dataset without the outliers
my_dataset = data_dict

## Processing Features

## Tuning Classifiers